In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
headers = {"X-Naver-Client-Id": "376ZADkaoIoxnJh_ixoG",
           "X-Naver-Client-Secret":"FEja8f5GwD"}


url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"",
    "ages":["50"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='50'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='50'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='50'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["디지털/가전"], axis=1, inplace=True)
df_all_a

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,5.08713,0.42870,2.85004,1.86086,14.68597,6.70634,0.96968,0.42830,1.21374,0.43404,...,4.72211,0.96968,0.55433,1.59224,0.63030,0.11935,0.10480,PC+MO,전체,50
2021-05-11,4.86217,0.38277,2.44830,1.59197,13.07411,6.38111,0.86514,0.39826,1.17342,0.40894,...,4.48593,0.86514,0.55940,1.49317,0.57676,0.11628,0.08290,PC+MO,전체,50
2021-05-12,4.96457,0.38918,2.30718,1.57181,12.76303,6.89432,0.83083,0.36114,1.19892,0.38450,...,4.43306,0.83083,0.56287,1.44591,0.60880,0.11602,0.08744,PC+MO,전체,50
2021-05-13,4.72211,0.38878,2.22427,1.70679,12.72030,9.38068,0.68063,0.31575,1.06460,0.42282,...,4.18874,0.68063,0.47409,1.41160,0.48290,0.09786,0.07783,PC+MO,전체,50
2021-05-14,4.49515,0.34031,2.04163,1.54738,11.81203,11.93486,0.73724,0.33804,1.07956,0.37409,...,4.02559,0.73724,0.44352,1.27916,0.49438,0.10373,0.08504,PC+MO,전체,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,7.20274,0.42647,2.55228,2.25616,12.74588,16.46350,0.73304,0.22578,0.96076,0.34569,...,4.09437,0.73304,0.58565,1.35752,0.56564,0.15529,0.11990,MO,전체,50
2023-06-30,6.42490,0.43528,2.37742,2.09877,11.75570,14.33516,0.70004,0.23563,0.90939,0.30805,...,3.82932,0.70004,0.55086,1.30227,0.50263,0.13080,0.11737,MO,전체,50
2023-07-01,8.06704,0.44439,2.85063,2.63306,13.83029,17.97424,0.86235,0.26042,0.88774,0.40751,...,4.44140,0.86235,0.63359,1.41785,0.67599,0.15709,0.15500,MO,전체,50


In [4]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"f",
    "ages":["50"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='50'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='50'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='50'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["디지털/가전"], axis=1, inplace=True)
df_all_b

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,3.93224,0.32591,1.67984,2.30028,19.29033,6.45919,0.48447,0.19341,0.29132,0.23678,...,2.70813,0.48447,0.41477,0.40971,0.39242,0.02234,0.08673,PC+MO,여성,50
2021-05-11,3.86572,0.30595,1.43747,2.00922,16.96930,6.17452,0.46372,0.17639,0.27988,0.23332,...,2.62220,0.46372,0.43047,0.37859,0.38098,0.03352,0.06571,PC+MO,여성,50
2021-05-12,3.96257,0.28387,1.40501,2.03077,16.62822,6.84816,0.44270,0.14659,0.29026,0.20140,...,2.50567,0.44270,0.38870,0.40865,0.35916,0.03724,0.06917,PC+MO,여성,50
2021-05-13,3.66911,0.29079,1.28343,2.17204,16.75725,9.53874,0.36715,0.14153,0.24290,0.21736,...,2.39685,0.36715,0.35411,0.37646,0.31739,0.02846,0.07130,PC+MO,여성,50
2021-05-14,3.68241,0.24769,1.13657,1.93525,15.18914,12.49590,0.36608,0.15031,0.24263,0.19607,...,2.30081,0.36608,0.33735,0.32458,0.28653,0.03964,0.06225,PC+MO,여성,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,6.08298,0.33136,1.58931,2.96905,16.28410,17.94852,0.39465,0.09620,0.25569,0.21237,...,2.16061,0.39465,0.44697,0.33811,0.27876,0.04416,0.07960,MO,여성,50
2023-06-30,5.40365,0.29226,1.45682,2.72264,14.94627,15.52517,0.33895,0.08073,0.21631,0.18481,...,1.89845,0.33895,0.41209,0.35752,0.25063,0.05091,0.08888,MO,여성,50
2023-07-01,6.87201,0.32714,1.72180,3.35021,17.23600,18.92658,0.43488,0.10942,0.20843,0.27285,...,2.28241,0.43488,0.45766,0.44163,0.34542,0.07313,0.11195,MO,여성,50


In [5]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
        ## 가전, 전자
        {"name": "생활가전", "param": [ "50001412"]},
        {"name": "영상가전", "param": [ "50000208"]},
        {"name": "이미용가전", "param": [ "50000211"]},
        {"name": "주방가전", "param": [ "50000213"]},
        {"name": "계절가전", "param": [ "50000212"]},
        {"name": "카메라/캠코더용품", "param": [ "50000152"]},
        
        ## 컴퓨터 및 주변기기
        {"name": "PC", "param": [ "50000089"]},
        {"name": "PC부품", "param": [ "50000097"]},
        {"name": "PC액세서리", "param": [ "50000090"]},
        {"name": "저장장치", "param": [ "50000096"]},
        {"name": "멀티미디어장비", "param": [ "50000095"]},
        {"name": "모니터", "param": [ "50000153"]},
        {"name": "모니터주변기기", "param": [ "50000093"]},
        {"name": "노트북", "param": [ "50000151"]},
        {"name": "노트북액세서리", "param": [ "50000091"]},
        {"name": "소프트웨어", "param": [ "50000099"]},
        {"name": "주변기기", "param": [ "50000094"]},
        
        ## 통신기기
        {"name": "음향가전", "param": [ "50000209"]},
        {"name": "태블릿PC", "param": [ "50000152"]},
        {"name": "태블릿PC액세서리", "param": [ "50000092"]},
        
        ## 기타
        {"name": "자동차기기", "param": [ "50000214"]},
        {"name": "게임기/타이틀", "param": [ "50000088"]},        
        {"name": "광학기기/용품", "param": [ "50000207"]},        
        {"name": "학습기기", "param": [ "50000087"]}]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
      {"name": "디지털/가전", "param": [ "50000003"]},
      {"name": "휴대폰액세서리", "param": [ "50000205"]},
    ],
    "device":i,
    "gender":"m",
    "ages":["50"]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "디지털/가전")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "휴대폰액세서리")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='50'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='50'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='50'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["디지털/가전"], axis=1, inplace=True)
df_all_c

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,5.77012,0.49133,3.71917,1.30924,9.27578,6.42078,1.34635,0.61380,1.97821,0.58411,...,6.23201,1.34635,0.64150,2.56950,0.80306,0.20039,0.11355,PC+MO,남성,50
2021-05-11,5.41510,0.42478,3.20012,1.08163,8.44724,6.08283,1.17193,0.57396,1.91413,0.54081,...,5.87427,1.17193,0.63631,2.41487,0.71449,0.18431,0.09252,PC+MO,남성,50
2021-05-12,5.51480,0.45719,2.96880,1.02423,8.18796,6.40742,1.12789,0.53290,1.95174,0.52523,...,5.88466,1.12789,0.68158,2.29933,0.79415,0.18035,0.09772,PC+MO,남성,50
2021-05-13,5.33840,0.45002,2.92823,1.14299,7.98880,8.51280,0.91983,0.45348,1.74689,0.58139,...,5.53311,0.91983,0.54922,2.26569,0.59969,0.15487,0.07793,PC+MO,남성,50
2021-05-14,4.90545,0.40029,2.72635,1.06778,7.76392,10.49596,1.02572,0.48663,1.77485,0.51088,...,5.32009,1.02572,0.50816,2.06851,0.64967,0.15536,0.09970,PC+MO,남성,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,7.87256,0.49649,3.38460,1.34715,8.11963,13.73165,1.03736,0.34618,1.63475,0.46157,...,5.83866,1.03736,0.69029,2.33422,0.82758,0.26126,0.15385,MO,남성,50
2023-06-30,7.04587,0.55507,3.17778,1.29478,7.56603,12.06612,1.02997,0.38168,1.57439,0.41542,...,5.59012,1.02997,0.65804,2.20433,0.73231,0.20563,0.13876,MO,남성,50
2023-07-01,8.75489,0.53614,3.83642,1.68919,9.26884,15.70134,1.25129,0.40092,1.53888,0.52045,...,6.39936,1.25129,0.77403,2.34428,0.97552,0.23433,0.18936,MO,남성,50


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7064 entries, 2021-05-10 to 2023-07-03
Data columns (total 28 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   휴대폰액세서리    7064 non-null   float64
 1   생활가전       7064 non-null   float64
 2   영상가전       7064 non-null   float64
 3   이미용가전      7064 non-null   float64
 4   주방가전       7064 non-null   float64
 5   계절가전       7064 non-null   float64
 6   카메라/캠코더용품  7064 non-null   float64
 7   PC         7064 non-null   float64
 8   PC부품       7064 non-null   float64
 9   PC액세서리     7064 non-null   float64
 10  저장장치       7064 non-null   float64
 11  멀티미디어장비    7064 non-null   float64
 12  모니터        7064 non-null   float64
 13  모니터주변기기    7064 non-null   float64
 14  노트북        7064 non-null   float64
 15  노트북액세서리    7064 non-null   float64
 16  소프트웨어      7064 non-null   float64
 17  주변기기       7064 non-null   float64
 18  음향가전       7064 non-null   float64
 19  태블릿PC      7064 non-null   float64
 20

In [7]:
## 엑셀에 저장
df_all.to_excel("MCLS_04_all_fm_50.xlsx", sheet_name="click ratio")
df_all.head()

,휴대폰액세서리,생활가전,영상가전,이미용가전,주방가전,계절가전,카메라/캠코더용품,PC,PC부품,PC액세서리,...,음향가전,태블릿PC,태블릿PC액세서리,자동차기기,게임기/타이틀,광학기기/용품,학습기기,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,,
2021-05-10,5.08713,0.42870,2.85004,1.86086,14.68597,6.70634,0.96968,0.42830,1.21374,0.43404,...,4.72211,0.96968,0.55433,1.59224,0.63030,0.11935,0.10480,PC+MO,전체,50
2021-05-11,4.86217,0.38277,2.44830,1.59197,13.07411,6.38111,0.86514,0.39826,1.17342,0.40894,...,4.48593,0.86514,0.55940,1.49317,0.57676,0.11628,0.08290,PC+MO,전체,50
2021-05-12,4.96457,0.38918,2.30718,1.57181,12.76303,6.89432,0.83083,0.36114,1.19892,0.38450,...,4.43306,0.83083,0.56287,1.44591,0.60880,0.11602,0.08744,PC+MO,전체,50
2021-05-13,4.72211,0.38878,2.22427,1.70679,12.72030,9.38068,0.68063,0.31575,1.06460,0.42282,...,4.18874,0.68063,0.47409,1.41160,0.48290,0.09786,0.07783,PC+MO,전체,50
2021-05-14,4.49515,0.34031,2.04163,1.54738,11.81203,11.93486,0.73724,0.33804,1.07956,0.37409,...,4.02559,0.73724,0.44352,1.27916,0.49438,0.10373,0.08504,PC+MO,전체,50
